In [8]:
import numpy as np
from PIL import Image
import os
from abc import ABC, abstractmethod
import random
from scipy.optimize import differential_evolution
from typing import List, Tuple, Callable, Any
import json


# Функция MSE

In [9]:
def mse(image1: np.ndarray, image2: np.ndarray) -> float:
    """Вычисление среднеквадратичной ошибки между двумя изображениями."""
    return np.mean((image1 - image2) ** 2)

In [10]:
def load_images_from_folder(folder_path: str) -> List[np.ndarray]:
    """Загрузка изображений из папки.
    
    Args:
        folder_path (str): Путь к папке с изображениями
        
    Returns:
        List[np.ndarray]: Список загруженных изображений
    """
    images = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            try:
                img = Image.open(file_path)
                images.append(np.array(img))
            except Exception as e:
                print(f"Не удалось загрузить изображение {filename}: {e}")
    return images

# Оптимизатор

In [11]:
class Optimizer(ABC):
    """Базовый класс для оптимизаторов"""
    def __init__(self, method: str):
        self.method = method

    @abstractmethod
    def optimize(self, objective_function: Callable, bounds: List[Tuple[float, float]], **kwargs) -> Tuple[Any, float]:
        """Абстрактный метод оптимизации"""
        pass

class MonteCarloOptimizer(Optimizer):
    """Оптимизатор методом Монте-Карло"""
    def __init__(self):
        super().__init__("monte_carlo")

    def optimize(self, objective_function: Callable, bounds: List[Tuple[float, float]], 
                max_iterations: int = 1000) -> Tuple[List[float], float]:
        best_point = None
        best_value = float('inf')
        
        for _ in range(max_iterations):
            point = [random.uniform(b[0], b[1]) for b in bounds]
            value = objective_function(point)
            if value < best_value:
                best_value = value
                best_point = point
                
        return best_point, best_value

class GridSearchOptimizer(Optimizer):
    """Оптимизатор методом перебора по сетке"""
    def __init__(self):
        super().__init__("grid_search")

    def optimize(self, objective_function: Callable, bounds: List[Tuple[float, float]], 
                steps: int = 10) -> Tuple[List[float], float]:
        best_point = None
        best_value = float('inf')
        
        grids = [np.linspace(b[0], b[1], steps) for b in bounds]
        for point in np.array(np.meshgrid(*grids)).T.reshape(-1, len(bounds)):
            value = objective_function(point)
            if value < best_value:
                best_value = value
                best_point = point
                
        return best_point, best_value

class PeanoOptimizer(Optimizer):
    """Оптимизатор на основе дифференциальной эволюции"""
    def __init__(self):
        super().__init__("peano")

    def optimize(self, objective_function: Callable, bounds: List[Tuple[float, float]], 
                max_iterations: int = 100) -> Tuple[List[float], float]:
        result = differential_evolution(
            objective_function,
            bounds,
            maxiter=max_iterations,
            disp=False
        )
        return result.x, result.fun



# Менеджер расчетов

In [12]:
class CalculationManager:
    """Менеджер для управления и сохранения результатов оптимизации"""
    __instance = None

    def __new__(cls, *args, **kwargs):
        if cls.__instance is None:
            cls.__instance = super(CalculationManager, cls).__new__(cls)
        return cls.__instance

    def __init__(self):
        if not hasattr(self, 'initialized'):
            self.results = []
            self.initialized = True

    def save_result(self, method_info: dict, best_point: List[float], best_accuracy: float) -> None:
        """Сохранение результата оптимизации"""
        result = {
            "method": method_info["method"],
            "bounds": method_info["bounds"],
            "best_point": self._convert_to_serializable(best_point),
            "best_accuracy": float(best_accuracy)  # Преобразуем в Python float
        }
        self.results.append(result)
        with open("results.json", "w") as f:
            json.dump(self.results, f, indent=4)

    def get_results(self) -> List[dict]:
        """Получение всех сохраненных результатов"""
        return self.results
    
    def _convert_to_serializable(self, obj):
        """Преобразование объектов в сериализуемый формат."""
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, (list, tuple)):
            return [self._convert_to_serializable(item) for item in obj]
        elif isinstance(obj, dict):
            return {key: self._convert_to_serializable(value) for key, value in obj.items()}
        return obj

In [13]:
def create_objective_function(template: np.ndarray, image: np.ndarray) -> Callable:
    """Создание целевой функции для оптимизации.
    
    Args:
        template (np.ndarray): Шаблонное изображение
        image (np.ndarray): Изображение для сравнения
        
    Returns:
        Callable: Целевая функция
    """
    def objective(x: List[float]) -> float:
        # Здесь можно добавить трансформации изображения
        # Например, масштабирование, поворот и т.д.
        transformed_image = image  # Пока без трансформаций
        return mse(template, transformed_image)
    return objective

In [14]:


# Загрузка изображений
folder_path = "save_folder"
images = load_images_from_folder(folder_path)

if not images:
    print("Не удалось загрузить изображения")
    exit(1)

# Используем первое изображение как шаблон
template = images[0]

# Определение границ параметров
bounds = [
    (0.5, 2.0),    # масштаб по X
    (0.5, 2.0),    # масштаб по Y
    (-45, 45),     # угол поворота
    (-50, 50),     # смещение по X
    (-50, 50)      # смещение по Y
]

# Создание менеджера расчетов
manager = CalculationManager()

# Тестирование разных оптимизаторов
optimizers = [
    (MonteCarloOptimizer(), {"method": "monte_carlo", "bounds": bounds}),
    (GridSearchOptimizer(), {"method": "grid_search", "bounds": bounds}),
    (PeanoOptimizer(), {"method": "peano", "bounds": bounds})
]

# Оптимизация для каждого изображения
for img in images[1:]:  # Пропускаем шаблон
    print(f"\nОбработка изображения...")
    
    for optimizer, method_info in optimizers:
        print(f"Тестирование {method_info['method']}")
        
        # Создание целевой функции
        objective = create_objective_function(template, img)
        
        # Запуск оптимизации
        best_point, best_accuracy = optimizer.optimize(
            objective_function=objective,
            bounds=bounds
        )
        
        # Сохранение результатов
        manager.save_result(method_info, best_point, best_accuracy)
        
        print(f"Лучшие параметры: {best_point}")
        print(f"Лучшее MSE: {best_accuracy}")

# Вывод всех результатов
print("\nВсе результаты:")
results = manager.get_results()
for result in results:
    print(f"\nМетод: {result['method']}")
    print(f"Лучшая точка: {result['best_point']}")
    print(f"Лучшая точность: {result['best_accuracy']}")


Обработка изображения...
Тестирование monte_carlo
Лучшие параметры: [1.0593027407594315, 1.8896782167175956, 30.006312937386568, -9.952168956319397, 12.538234648981508]
Лучшее MSE: 43.81089626736111
Тестирование grid_search
Лучшие параметры: [  0.5   0.5 -45.  -50.  -50. ]
Лучшее MSE: 43.81089626736111
Тестирование peano
Лучшие параметры: [  1.63306689   1.33210109 -33.29156526  32.84098888 -45.11961973]
Лучшее MSE: 43.81089626736111

Обработка изображения...
Тестирование monte_carlo
Лучшие параметры: [1.7875190725182883, 0.6580650990087391, 29.185929726063563, 37.72732096842958, -33.59113479761402]
Лучшее MSE: 52.571771918402774
Тестирование grid_search
Лучшие параметры: [  0.5   0.5 -45.  -50.  -50. ]
Лучшее MSE: 52.571771918402774
Тестирование peano
Лучшие параметры: [  1.89930292   1.63736744  31.85986395  45.02256974 -30.4371146 ]
Лучшее MSE: 52.571771918402774

Обработка изображения...
Тестирование monte_carlo
Лучшие параметры: [0.6460854758542022, 1.171314706777443, -6.80100452